##    Trabajo práctico Nro 1

## **Procesamiento de Lenguaje Natural**

## Integrantes:

*   Kidonakis, Sol
*   Leguiza, Claudia E



###                  Año: 2024

In [ ]:
import requests
import csv
import pandas as pd
import lxml


In [ ]:
# Dirección de la página web
url = "https://ww3.lectulandia.com/"

In [ ]:
# Ejecutar GET-Request
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup

 Cargamos el texto y localizamos los datos que necesitamos para armar el archivo CSV

In [ ]:
html = BeautifulSoup(response.text, 'html.parser')

In [ ]:
genero_html = html.find_all("a", class_ = "term")
genero_html

In [ ]:
resumen_html = html.find_all('div', class_="description")
resumen_html

In [ ]:
titulo_html = html.find_all('a', class_="title")
titulo_html

In [ ]:
autor_html = html.find_all('div', class_="subdetail")
autor_html

Definimos los 10 generos que vamos a cargar

### Scraping

Se realiza el parseo del contenido deseado según las etiquetas html del sitio.

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))
stop_words = list(stop_words)


def scrape_lectulandia():

    base_url = "https://ww3.lectulandia.com/genero/"
    generos = ['cuentos', 'humor', 'infantil', 'medicina', 'musica','poesia',\
               'politica', 'sociologia','tecnologia', 'viajes']
    libros = []

    for genero in generos:
            url = f"{base_url}{genero}"
            response = requests.get(url)
            if response.status_code != 200:
                break  # Rompe el bucle si la página no existe
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer los elementos de los libros

            titulos = soup.find_all('a', class_='title')
            autores = soup.find_all('div', class_='subdetail')
            resumenes = soup.find_all('div', class_='description')

            for titulo, autor, resumen in zip(titulos, autores, resumenes):

                libro_titulo = titulo.get_text(strip=True)
                libro_autor = autor.get_text(strip=True)
                libro_genero = genero  # Usamos el género que estamos iterando
                libro_resumen = resumen.get_text(strip=True)

                libros.append({
                    'titulo': libro_titulo,
                    'autor': libro_autor,
                    'genero': libro_genero,
                    'sinopsis': libro_resumen
                })

    return libros


In [ ]:
libros = scrape_lectulandia()
df = pd.DataFrame(libros)

Verificamos la carga correcta de los generos seleccionados y el balanceo del dataset.

In [ ]:
df['genero'].unique()

In [ ]:
df['genero'].value_counts()

El dataset **df** se guarda un archivo **libros_lectulandia.csv**

In [ ]:
df.to_csv('libros_lectulandia.csv', index=False)

**Carga del archivo**

In [ ]:
libros = pd.read_csv('libros_lectulandia.csv')
libros.sample(10)

In [ ]:
libros['sinopsis'][101]


**Eliminacion de acentos,puntuaciones,mayusculas y espacios en blanco**

In [ ]:
import re
import unicodedata

def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

    # Eliminar puntuación
    texto = re.sub(r'[^\w\s]', '', texto)

    # Eliminar espacios en blanco adicionales
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto


**Recomendacion directa**

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

# Función para tokenizar las sinopsis
def tokenizar_sinopsis(sinopsis):
    return word_tokenize(sinopsis.lower())

# Tokenizar las sinopsis y entrenar el modelo Word2Vec
textos_tokenizados = [tokenizar_sinopsis(sinopsis) for sinopsis in df['sinopsis']]
modelo_word2vec = Word2Vec(sentences=textos_tokenizados, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def recomendacion_directa_word2vec(entrada_usuario, df, modelo_word2vec, num_recomendaciones=3):
    # Preprocesar la entrada del usuario
    entrada_procesada = preprocesar_texto(entrada_usuario)

    # Calcular la similitud entre la entrada del usuario y las sinopsis de los libros usando Word2Vec
    similitudes = {}
    for idx, row in df.iterrows():
        similitudes[idx] = modelo_word2vec.wv.n_similarity(entrada_procesada.split(), row['sinopsis'].split())

    # Ordenar por similitud descendente
    similitudes_ordenadas = sorted(similitudes.items(), key=lambda x: x[1], reverse=True)
    libros_relevantes = df.iloc[[idx for idx, _ in similitudes_ordenadas[:num_recomendaciones]]]

    return libros_relevantes


**Recomendacion por autor**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
# Función para recomendar libros por autor
def recomendar_libros_por_autor(autor):
    # Preprocesar el nombre del autor
    autor_procesado = preprocesar_texto(autor)

    # Calcular la similitud coseno entre el nombre del autor y todos los autores en el DataFrame
    tfidf_vectorizer = TfidfVectorizer()
    autor_vector = tfidf_vectorizer.fit_transform([autor_procesado])
    autores_vectorizados = tfidf_vectorizer.transform(df['autor'])
    similitudes = cosine_similarity(autor_vector, autores_vectorizados)

    # Obtener los índices de los dos autores más similares
    indices_autores_similares = similitudes.argsort(axis=1)[0][-2:]

    # Obtener los libros asociados a los dos autores más similares
    libros_relevantes = df[df.index.isin(indices_autores_similares)].copy()

    # Seleccionar los dos libros más relevantes del autor más similar
    libros_autor_mas_similar = libros_relevantes.iloc[0:2]

    # Seleccionar uno de los libros del segundo autor más cercano al azar si hay varios
    libros_segundo_autor = libros_relevantes.iloc[2:]
    if len(libros_segundo_autor) > 0:
        libro_elegido = libros_segundo_autor.sample(1)
        libros_recomendados = pd.concat([libros_autor_mas_similar, libro_elegido])
    else:
        libros_recomendados = libros_autor_mas_similar

    return libros_recomendados



**Recomendacion por Genero Literario**

In [ ]:
# Función para recomendar libros por género literario
def recomendar_libros_por_genero(genero):
    # Preprocesar el género literario
    genero_procesado = preprocesar_texto(genero)

    # Calcular la similitud coseno entre el género literario y todos los géneros en el DataFrame
    tfidf_vectorizer = TfidfVectorizer()
    genero_vector = tfidf_vectorizer.fit_transform([genero_procesado])
    generos_vectorizados = tfidf_vectorizer.transform(df['genero'])
    similitudes = cosine_similarity(genero_vector, generos_vectorizados)

    # Obtener los índices de los géneros más similares
    indices_generos_similares = similitudes.argsort(axis=1)[0][-2:]

    # Obtener los libros asociados a los géneros más similares
    libros_relevantes = df[df.index.isin(indices_generos_similares)].copy()

    return libros_relevantes


**Intraccion con el usuario**

In [ ]:
# Mostrar recomendaciones
def mostrar_recomendaciones(recomendaciones):
    for idx, libro in enumerate(recomendaciones.iterrows(), start=1):
        print(f"{idx}. Título: {libro[1]['titulo']}")
        print(f"   Autor: {libro[1]['autor']}")
        print(f"   Género: {libro[1]['genero']}")
        print(f"   Sinopsis: {libro[1]['sinopsis']}\n")

In [ ]:
def interactuar_con_usuario(df,modelo_word2vec):
    print("¡Bienvenido al recomendador de libros de Lectulandia!")
    print("¿Qué tienes ganas de leer hoy?")
    print("1. Recomendación Directa")
    print("2. Elección por Genero Literario")
    print("3. Elección por Autor")
    opcion = input("Seleccione una opción (1/2/3): ")

    if opcion == "1":
        entrada_usuario = input("Ingrese una temática o palabras clave: ")
        recomendaciones = recomendacion_directa_word2vec(entrada_usuario,df,modelo_word2vec)
    elif opcion == "2":
        genero_usuario = input("Ingrese el género literario: ")
        recomendaciones = recomendar_libros_por_genero(genero_usuario)
    elif opcion == "3":
        autor_usuario = input("Ingrese el autor: ")
        recomendaciones = recomendar_libros_por_autor(autor_usuario)
    else:
        print("Opción no válida. Por favor, seleccione una opción válida (1/2/3).")
        return

    if recomendaciones.empty:
        print("Lo siento, no se encontraron recomendaciones para tu búsqueda.")
    else:
        print("\nAquí tienes algunas recomendaciones para ti:\n")
        mostrar_recomendaciones(recomendaciones)






In [ ]:

# Ejecutar la interacción con el usuario
interactuar_con_usuario(df, modelo_word2vec)